In [6]:
import msnoise as ms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_ini = 'D:\\MEMOIRE\\MSNoise\\ablation\\db.ini'
database = ms.connect(inifile=path_ini)

In [ ]:
ms.api.read_db_inifile(inifile=path_ini)

In [5]:
ms.api.Config('startdate', '2019-01-01')

<msnoise.msnoise_table_def.declare_tables.<locals>.Config at 0x1fe06d11970>

In [14]:
ms.api.update_filter(database,ref=2,low=1.0,high=2.0,mwcs_low=1.0,mwcs_high=2.0, mwcs_wlen=120, 
                     mwcs_step=60 , rms_threshold=0, used=True)

In [15]:
ms.api.get_filters(database, all=True)

[<msnoise.msnoise_table_def.declare_tables.<locals>.Filter at 0x1fe06d1ad60>,
 <msnoise.msnoise_table_def.declare_tables.<locals>.Filter at 0x1fe06d1a910>]

In [18]:
stt = ms.api.get_stations(database, all=True)

In [20]:
ms.api.get_interstation_distance(stt[1],stt[2],coordinates='DEG')

19.366583213567928

In [23]:
ms.api.get_data_availability(database, net='SS')

[]